## 1.Sampling Task
Pick any of the CTU-13 datasets (Malware capture 42 to 54). Download the unidirectional netflows. DO NOT DOWNLOAD THE VIRUS THAT WAS USED TO GENERATE THE DATA UNLESS USING A VM OR OTHER SANDBOX. The flows are collected from a host in the network. Its IP address should be obvious from the data sample. We are interested in the other addresses the host connects with.
Estimate the distribution over the other IP_addresses, what are the 10 most frequent values? Write code for RESERVOIR sampling, use it to estimate the distribution in one pass (no need to actually stream the data, you may store it in memory, or run every file separately, but do store and load the intermediate results). Use a range of reservoir sizes. What are the 10 most frequent IP-addresses and their frequencies when sampled? Use the theory to explain any approximation errors you observe.

In [1]:
import pandas as pd
from mmh3 import hash
import math
import matplotlib
import numpy as np
import time
import operator
import collections

Dataset is CTU-Malware-Capture-Botnet-43. Infected host's ip is 147.32.84.165

In [2]:
#pre-process, divide raw data in to follwing coulmns: Data, Duration, Protocol, Source, Destination and Label
columns=['Date','Duration','Protocol','Source','Dest','Label']
lst=[]
with open("capture20110811.pcap.netflow.labeled") as fp:  
    for cnt, line in enumerate(fp):
        if cnt!=0:
            dat=line.split("\t")
            lst.append([dat[0],dat[1],dat[2],dat[3].split(':')[0],dat[5].split(':')[0],dat[11].split('\n')[0]])
dataset=pd.DataFrame(lst, columns=columns)

Filter out the other addresses connected to infected host

In [3]:
infected_host='147.32.84.165'
infected_dataset=dataset.loc[(dataset['Source']==infected_host) | (dataset['Dest']==infected_host)]
infected_dataset=infected_dataset.reset_index()
infected_dataset.to_csv('infected_dataset.csv')
infected_dataset.head()

,index,Date,Duration,Protocol,Source,Dest,Label
0,540283,2011-08-11 10:27:20.087,0.000,UDP,147.32.84.165,147.32.80.9,1
1,541362,2011-08-11 10:27:22.334,0.000,UDP,147.32.84.165,147.32.80.9,1
2,541377,2011-08-11 10:27:22.355,0.045,TCP,147.32.84.165,74.125.232.198,Botnet
3,541384,2011-08-11 10:27:22.362,0.034,TCP,74.125.232.198,147.32.84.165,Botnet
4,702906,2011-08-11 10:32:25.092,0.000,UDP,147.32.84.165,147.32.80.9,1


Calculate the group truth distribution

In [4]:
from collections import Counter
#exclude host machine
counter=Counter(list(infected_dataset[infected_dataset['Dest']!='147.32.84.165'].loc[:,'Dest']))
top10=counter.most_common(10)
top10_percentage=[]
for i in range(len(top10)):
    top10_percentage.append([top10[i][0],top10[i][1]/len(infected_dataset)])
top10_percentage

[['193.23.181.44', 0.11903337258005366],
 ['174.128.246.102', 0.06614680846957093],
 ['174.37.196.55', 0.06479569186820823],
 ['67.19.72.206', 0.0605107220753151],
 ['72.20.15.61', 0.05724874056631087],
 ['173.236.31.226', 0.03296724507324982],
 ['184.154.89.154', 0.032388195101237235],
 ['46.4.36.120', 0.031403810148815846],
 ['147.32.80.9', 0.015190410932463472],
 ['217.163.21.37', 0.013530467679360728]]

In [5]:
#In order to save space, process file without read whole dataset at once. This function is not used 
def RESERVIOR(fp,m):
    #fp is file pointer; m is the reservior size
    reservior=[]
    count=0
    while True:
        newline=fp.readline()
        if not newline:
            break
        else:
            dat=newline.split("\t")
            Source=dat[3].split(':')[0].strip()
            Dest=dat[5].split(':')[0].strip()
            #filter data
            if Dest==infected_host or Source!=infected_host:
                continue
        if count<m: 
            reservior.append(Dest)
        else:
            #Choose to sample the i’th item (i>m) with probability pi = m/i
            if np.random.uniform(0,1)<m/count:
                s=np.random.randint(0,m)
                reservior[s]=Dest
        count=count+1
    return reservior

Test reservior algorithm using size 10,100,1000. This process can be very time consuming

In [6]:
timeRESERVIOR=[]
for j in [10,100,1000,10000]:
    start_time = time.time()
    with open("capture20110811.pcap.netflow.labeled") as fp:
        fp.readline()
        sampledData=RESERVIOR(fp,j)
        counter=Counter(sampledData)
        top10=counter.most_common(10)
        top10_percentage=[]
        for i in range(len(top10)):
            top10_percentage.append([top10[i][0],top10[i][1]/len(sampledData)])
            timeCount = time.time() - start_time
            timeRESERVIOR.append(timeCount)
        print(f'distribution when k={j} is: {top10_percentage} with time {timeCount} s \n')

print(f"averaged time is {np.mean(timeRESERVIOR)} s")

distribution when k=10 is: [['174.37.196.55', 0.2], ['217.163.21.37', 0.1], ['196.218.117.248', 0.1], ['67.19.72.206', 0.1], ['174.128.246.102', 0.1], ['92.240.244.181', 0.1], ['46.4.36.120', 0.1], ['205.188.190.2', 0.1], ['67.195.168.31', 0.1]] with time 37.68745756149292 s 

distribution when k=100 is: [['193.23.181.44', 0.1], ['72.20.15.61', 0.09], ['67.19.72.206', 0.08], ['174.128.246.102', 0.07], ['174.37.196.55', 0.07], ['173.236.31.226', 0.06], ['184.154.89.154', 0.04], ['74.6.136.244', 0.03], ['46.4.36.120', 0.02], ['94.100.28.114', 0.02]] with time 13.108110666275024 s 

distribution when k=1000 is: [['193.23.181.44', 0.129], ['174.128.246.102', 0.078], ['174.37.196.55', 0.076], ['67.19.72.206', 0.065], ['72.20.15.61', 0.059], ['173.236.31.226', 0.037], ['46.4.36.120', 0.037], ['184.154.89.154', 0.037], ['94.100.28.114', 0.021], ['147.32.80.9', 0.018]] with time 12.791190385818481 s 

distribution when k=10000 is: [['193.23.181.44', 0.1354], ['174.128.246.102', 0.0745], ['174.

## 2.Sketching task
Build code for computing a COUNT-MIN sketch, play with different heights and widths for the Count-Min sketch matrix. Compare it to the RESERVOIR sampling strategy. Is it more space-efficient/accurate? What about run-time? Use the theory to explain any differences you observe

In [8]:
def mostFrequent(dataset, k):
    IPs=[]
    for i in range (0,len(dataset)):
        if (dataset.Source[i] == infected_host):
            IPs.append(dataset.Dest[i])
        else:
            IPs.append(dataset.Source[i])
    counter=collections.Counter(IPs)
    return(counter.most_common(k), IPs)
most_freq,ipList = mostFrequent(infected_dataset, 10)

In [9]:
#count min algorithm
class Sketching:
    def __init__(self, size, hash_count):
        self.size = size
        self.hash_count = hash_count
        self.sketch_array = np.zeros((hash_count,size),dtype=np.int)
        
    def add(self, string):
        for seed in range(self.hash_count):
            result = hash(string, seed) % self.size
            self.sketch_array[seed][result]+=1
            
    def estimate(self, string):
        minimum = 1000000
        for seed in range(self.hash_count):
            result = hash(string, seed) % self.size
            minimum = min(minimum,self.sketch_array[seed][result])
        return minimum

In [12]:
import mmh3
#test on different width and height
for e,d in [(0.002,np.exp(-1)) , (0.002,np.exp(-10)) , (0.0002,np.exp(-1)) , (0.0002,np.exp(-10)) , (0.00002,np.exp(-1)) , (0.00002,np.exp(-10))]:
        startTime=time.time()
        width = int(2 / e)  
        height = int(np.log(1/d)) 
        print(f"width is {width} , height is {height}")
        #construct matrix
        countMin = Sketching(int(width), int(height))
        for ip in ipList:
                countMin.add(ip)
        estimation = {}
        for ip in ipList:
            estimation[ip] = countMin.estimate(ip)
        # estimate top 10
        top10Est = sorted(estimation.items(), key=operator.itemgetter(1), reverse = True)
        error = 0
        for i in range(10):
            #difference between ground truth and estimation
            error+= abs(top10Est[i][1] - most_freq[i][1])
        print(f"time is {time.time()-startTime} s")
        print(f" accuracy is  {100-(error*100.0/len(ipList))} %")
        print('-------------------------')

width is 1000 , height is 1
time is 0.2300732135772705 s
 accuracy is  87.93066841668436 %
-------------------------
width is 1000 , height is 10
time is 1.2771072387695312 s
 accuracy is  99.99806983342663 %
-------------------------
width is 10000 , height is 1
time is 0.21059322357177734 s
 accuracy is  100.0 %
-------------------------
width is 10000 , height is 10
time is 1.2811062335968018 s
 accuracy is  100.0 %
-------------------------
width is 99999 , height is 1
time is 0.20862531661987305 s
 accuracy is  100.0 %
-------------------------
width is 99999 , height is 10
time is 1.2736387252807617 s
 accuracy is  100.0 %
-------------------------
